# Plano de Teste

### Testes na camada trusted para os dados sobre de internet móvel.
Testes realizados após transformar os dataset de formato shapefile em csv


In [1]:
#importando bibliotecas necessárias para os testes
import pandas as pd
import great_expectations as gx 

#Criar o df tax_df com as informacoes doa arquivo us_tax_data_2016.csv
path ='./dados/validacoes_trusted/trusted_speedtest_mobile.csv'
df_mobile = pd.read_csv(path, delimiter=';', encoding="ansi")

In [2]:
# verificacao do layout do dataset
print(df_mobile.head(2))


           quadkey  avg_d_kbps  avg_u_kbps  avg_lat_ms  tests  devices   
0  323230233223102       15591       17249          26      4        2  \
1  323230233232230       16998       20375          25      2        1   

   index_righ   CD_MUN  latitude  longitude        date  quarter network_ty   
0         138  1400100  2.847033 -60.751648  2022-01-01        1     mobile  \
1         138  1400100  2.819601 -60.718689  2022-01-01        1     mobile   

                                            geometry  
0  POLYGON ((-60.75439453125 2.8497764266656, -60...  
1  POLYGON ((-60.721435546875 2.82234424689409, -...  


Crie uma expectation para utilizar no great expectation para relizações de algumas validações:

In [3]:
df_gx = gx.dataset.PandasDataset(df_mobile)
df_gx.head(3)

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,index_righ,CD_MUN,latitude,longitude,date,quarter,network_ty,geometry
0,323230233223102,15591,17249,26,4,2,138,1400100,2.847033,-60.751648,2022-01-01,1,mobile,"POLYGON ((-60.75439453125 2.8497764266656, -60..."
1,323230233232230,16998,20375,25,2,1,138,1400100,2.819601,-60.718689,2022-01-01,1,mobile,"POLYGON ((-60.721435546875 2.82234424689409, -..."
2,323232011001023,17268,8038,29,2,2,138,1400100,2.792168,-60.768127,2022-01-01,1,mobile,"POLYGON ((-60.7708740234375 2.7949114196117, -..."


``1`` Verifica se o código município (CD_MUN) contém apenas valores únicos.

Usar a função ``expect_column_values_to_be_unique`` para definir essa expectativa

In [4]:
validation_result = df_gx.expect_column_values_to_be_unique(column = 'CD_MUN')
validation_result

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 588373,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 588342,
    "unexpected_percent": 99.99473123341825,
    "unexpected_percent_total": 99.99473123341825,
    "unexpected_percent_nonmissing": 99.99473123341825,
    "partial_unexpected_list": [
      1400100,
      1400100,
      1400100,
      1400100,
      1400100,
      1400100,
      1400100,
      1400100,
      1400100,
      1400100,
      1400100,
      1400100,
      1400100,
      1400100,
      1400100,
      1400100,
      1400100,
      1400100,
      1400100,
      1400100
    ]
  },
  "meta": {},
  "success": false
}

Resultado da validação.

In [5]:
if validation_result['success']:
    print('A coluna de código de município contem apenas valores únicos')
else:
    print('A coluna código de município possui valores duplicados')

A coluna código de município possui valores duplicados


``2`` Validação de valores nulos em uma coluna


In [6]:
validation_result = df_gx.expect_column_values_to_not_be_null(column= 'CD_MUN')
validation_result

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 588373,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "success": true
}

In [7]:
if validation_result['success']:
    print('A coluna código do município não possui valores nulos')
else:
    print('A coluna código do município possui valores nulos, verificar a necessidade de realizar tratamento no campo')

A coluna código do município não possui valores nulos


``3`` Verifica se há valores inválidos para latitude.


In [8]:
latitude_invalida = df_gx[(df_gx['latitude'] < -90) | (df_gx['latitude'] > 90)]

if latitude_invalida.empty:
    print("Não há valores incorretos na coluna latitude.")
else:
    print("Existe(m) valor(es) incorreto(s) na coluna latitude:")
    print('Valo(res) invalido(s):')
    print(latitude_invalida['latitude'].to_string(index=False))

Não há valores incorretos na coluna latitude.


In [9]:
null_latitude_count = latitude_invalida['latitude'].count()
print("Quantidade de valor(es) de latitude incorreto(s):", null_latitude_count)


Quantidade de valor(es) de latitude incorreto(s): 0


In [10]:
longitude_invalida = df_gx[(df_gx['longitude'] < -180) | (df_gx['longitude'] > 180)]

if longitude_invalida.empty:
    print("Não há valores incorretos na coluna longitude.")
else:
    print("Existe(m) valor(es) incorreto(s) na coluna longitude:")
    print(longitude_invalida['longitude'].to_string(index=False))


Não há valores incorretos na coluna longitude.


``4`` Verifica se há datas incorretas.

In [11]:
from datetime import datetime

data_invalida = []

for valor_data in df_gx['date']:
    try:
        datetime.strptime(valor_data, '%Y-%m-%d')
    except ValueError:
        data_invalida.append(valor_data)

if len(data_invalida) == 0:
    print("Não há valores incorretos na coluna de datas.")
else:
    # Criar DataFrame com os valores incorretos
    df_data_invalida = pd.DataFrame({'Data(s) Incorreta(s):': data_invalida})
    print("Existe(m) valor(es) incorreto(s) na coluna de datas:")
    print(df_data_invalida.to_string(index=False))


Não há valores incorretos na coluna de datas.
